In [19]:
from others.module import *
import torch
import torch.nn as nn
from torch.autograd import Variable

In [20]:
randomStuff = torch.rand(5, 3, 16, 16)

# conv test

In [21]:
kernel = 2
stride = 3
padding = 1
in_channels = 3
out_channels =48
randomStuff = torch.rand(10, in_channels, 25, 25)
ourConv = Conv2d(in_channels, out_channels, kernel, stride, padding)
torchConv = nn.Conv2d(in_channels, out_channels, kernel, stride, padding)

In [22]:

ourConv.weights = torchConv.weight
ourConv.bias = torchConv.bias

In [23]:
ourOutput = ourConv.forward(randomStuff)
torchOutput = torchConv(randomStuff)
torch.testing.assert_allclose(torchOutput, ourOutput)

In [24]:
x=Variable(randomStuff,requires_grad=True)
x2=Variable(x*1.0,requires_grad=True)

In [25]:
x.grad

In [26]:
o=ourConv.forward(x2)

In [27]:
p=torchConv(x)

In [28]:
grdw=torch.rand(p.shape)

In [29]:


p.backward(grdw)

In [30]:
a=ourConv.backward(grdw)

In [31]:
torch.testing.assert_allclose(a, x.grad)

In [32]:
torch.testing.assert_allclose(torchConv.weight.grad,ourConv.dweights)


In [33]:
torch.testing.assert_allclose(torchConv.bias.grad,ourConv.dbias)

In [35]:
torchConv.bias.grad

tensor([417.8065, 402.0136, 402.9307, 396.2509, 428.6507, 409.7196, 394.5825,
        410.5635, 412.7366, 407.8360, 405.2646, 407.4084, 401.7672, 402.5669,
        400.5905, 397.1891, 407.7005, 387.9539, 391.0354, 401.2856, 389.5426,
        405.0171, 421.5373, 407.3176, 401.0757, 413.8104, 407.7632, 400.5075,
        385.0281, 406.4738, 399.6791, 411.9471, 401.3462, 410.1255, 399.9250,
        402.8688, 394.9614, 404.8026, 407.2644, 416.3716, 401.6061, 397.4891,
        400.5943, 403.0455, 406.2382, 405.8355, 393.1065, 405.1621])

In [34]:
ourConv.dbias

tensor([417.8066, 402.0137, 402.9308, 396.2509, 428.6508, 409.7197, 394.5824,
        410.5634, 412.7368, 407.8359, 405.2647, 407.4084, 401.7672, 402.5669,
        400.5906, 397.1893, 407.7006, 387.9539, 391.0354, 401.2855, 389.5424,
        405.0171, 421.5372, 407.3176, 401.0758, 413.8102, 407.7631, 400.5074,
        385.0281, 406.4738, 399.6792, 411.9472, 401.3464, 410.1255, 399.9251,
        402.8688, 394.9614, 404.8026, 407.2645, 416.3716, 401.6061, 397.4892,
        400.5943, 403.0456, 406.2380, 405.8353, 393.1066, 405.1621])

# upsample test

In [3]:
scale_factor = 2
loss=nn.MSELoss()
ourUpsample = NearestUpsampling(scale_factor=scale_factor)
torchUpsample = nn.Upsample(scale_factor = scale_factor)

In [8]:
ourOutput = ourUpsample.forward(randomStuff)
torchOutput = torchUpsample(randomStuff)
torch.testing.assert_allclose(torchOutput, ourOutput)

In [5]:
x=Variable(torch.rand(5*3*16).view((5,3,4,4))*1.0,requires_grad=True)

In [6]:
outp=torch.rand(5*3*8*8).view(5,3,8,8)

In [7]:
grdout=torch.rand_like(torchUpsample(x))

In [23]:
from torch.autograd import Variable

In [9]:
torchUpsample(x).backward(grdout)

In [12]:
torch.testing.assert_allclose(ourUpsample.backward(grdout),x.grad)

# MSE test

In [60]:
ourMSE = MSE()
torchMSE = nn.MSELoss()

In [61]:
x=Variable(randomStuff,requires_grad=True)

In [62]:
ourOutput = ourMSE.forward(x, torch.zeros_like(x))
torchOutput = torchMSE(x, torch.zeros_like(x))
torch.testing.assert_allclose(torchOutput, ourOutput)

In [63]:
#test not written correctly
ourBackward = ourMSE.backward()
torchBackward = torchOutput.backward()
torch.testing.assert_allclose(x.grad, ourBackward)

# ReLu test

In [97]:
ourReLU = ReLU()
torchReLU = nn.ReLU()
x=Variable(randomStuff,requires_grad=True)

In [98]:
ourOutput = ourReLU.forward(x)
torchOutput = torchReLU(x)
torch.testing.assert_allclose(torchOutput, ourOutput)

In [99]:
grdout=torch.rand_like(x)

In [104]:
grdout.shape

torch.Size([10, 3, 25, 25])

In [100]:
torchOutput.backward(grdout)

In [101]:
torch.testing.assert_allclose(ourReLU.backward(grdout),x.grad)

In [102]:
ourReLU.backward(grdout).shape

torch.Size([10, 3, 25, 25])

# sigmoid test

In [107]:
ourSigmoid = Sigmoid()
torchSigmoid = nn.Sigmoid()
x=Variable(randomStuff,requires_grad=True)
grdout=torch.rand_like(x)

In [108]:
ourOutput = ourSigmoid.forward(x)
torchOutput = torchSigmoid(x)
torch.testing.assert_allclose(torchOutput, ourOutput)

In [109]:
grdout.shape

torch.Size([10, 3, 25, 25])

In [110]:
torchOutput.backward(grdout)

In [111]:
torch.testing.assert_allclose(ourSigmoid.backward(grdout),x.grad)

In [112]:
ourSigmoid.backward(grdout).shape

torch.Size([10, 3, 25, 25])

# Sequential test

In [8]:
ourSequential = Sequential(ourConv, ReLU(), Sigmoid())
torchSequential = nn.Sequential( torchConv, nn.ReLU(), nn.Sigmoid())
x=Variable(randomStuff,requires_grad=True)

In [28]:
ourSequential = Sequential(ourConv)
torchSequential = nn.Sequential( torchConv)
x=Variable(randomStuff,requires_grad=True)

In [9]:
ourOutput = ourSequential.forward(x)
torchOutput = torchSequential(x)
torch.testing.assert_allclose(torchOutput, ourOutput)

In [10]:
grdout=torch.rand_like(ourOutput)

In [11]:
torchOutput.backward(grdout)

In [12]:
grdout.shape

torch.Size([10, 48, 9, 9])

In [13]:
ourSequential.backward(grdout)

torch.Size([48, 9, 9])
torch.Size([48, 9, 9])
torch.Size([48, 9, 9])


In [14]:
stuff=ourConv.dweights

In [17]:
torch.testing.assert_allclose(torchConv.weight.grad,stuff)

In [18]:
torch.testing.assert_allclose(torchConv.bias.grad,ourConv.dbias)

In [16]:
torchConv.bias.grad/stuff

tensor([[[[1.0000, 1.0000],
          [1.0000, 1.0000]],

         [[1.0000, 1.0000],
          [1.0000, 1.0000]],

         [[1.0000, 1.0000],
          [1.0000, 1.0000]]],


        [[[1.0000, 1.0000],
          [1.0000, 1.0000]],

         [[1.0000, 1.0000],
          [1.0000, 1.0000]],

         [[1.0000, 1.0000],
          [1.0000, 1.0000]]],


        [[[1.0000, 1.0000],
          [1.0000, 1.0000]],

         [[1.0000, 1.0000],
          [1.0000, 1.0000]],

         [[1.0000, 1.0000],
          [1.0000, 1.0000]]],


        [[[1.0000, 1.0000],
          [1.0000, 1.0000]],

         [[1.0000, 1.0000],
          [1.0000, 1.0000]],

         [[1.0000, 1.0000],
          [1.0000, 1.0000]]],


        [[[1.0000, 1.0000],
          [1.0000, 1.0000]],

         [[1.0000, 1.0000],
          [1.0000, 1.0000]],

         [[1.0000, 1.0000],
          [1.0000, 1.0000]]],


        [[[1.0000, 1.0000],
          [1.0000, 1.0000]],

         [[1.0000, 1.0000],
          [1.0000, 1.0000]],

    

In [95]:
torch.arange(3888)

tensor([   0,    1,    2,  ..., 3885, 3886, 3887])

# Full model

In [10]:
len((1,2))

2

In [26]:
(1,2)[1]

2